In [1]:
import csv
from enum import Enum
import numpy as np

import project_path
from model.data_accessor import get_data_file_abs_path
from model.connectome_loader import load_connectome_dict_cook, load_connectome_cook
from model.neuron_metadata import NeuronMetadataCollection

neuron_metadata_collection = NeuronMetadataCollection.load_from_chem_json(get_data_file_abs_path('chem.json'))
conn_spec_to_weight_gap, conn_spec_to_weight_chem = load_connectome_dict_cook()
cook_neurons = set()
for (source, target) in conn_spec_to_weight_gap.keys():
  cook_neurons.add(source)
  cook_neurons.add(target)
for (source, target) in conn_spec_to_weight_chem.keys():
  cook_neurons.add(source)
  cook_neurons.add(target)

# Neuron set differences

In [2]:
# Cook has 448 neurons, Varshney 279
varshney_neurons = neuron_metadata_collection.name_to_id.keys()
print("Cook has %d neurons, Varshney %d" % (len(cook_neurons), len(varshney_neurons)))

# All neurons in varshney found in cook
varshney_diff_cook = varshney_neurons - cook_neurons
print("In varshney, but not in cook: (%d) %s" % (len(varshney_diff_cook), varshney_diff_cook))

  # 169 neurons in cook are not found in varshney.
cook_diff_varshney = cook_neurons - varshney_neurons
print("In cook, but not in varshney: (%d) %s" % (len(cook_diff_varshney), sorted(cook_diff_varshney)))


Cook has 448 neurons, Varshney 279
In varshney, but not in cook: (0) set()
In cook, but not in varshney: (169) ['ANAL', 'DBWML1', 'DBWML10', 'DBWML11', 'DBWML12', 'DBWML13', 'DBWML14', 'DBWML15', 'DBWML16', 'DBWML17', 'DBWML18', 'DBWML19', 'DBWML2', 'DBWML20', 'DBWML21', 'DBWML22', 'DBWML23', 'DBWML24', 'DBWML3', 'DBWML4', 'DBWML5', 'DBWML6', 'DBWML7', 'DBWML8', 'DBWML9', 'DBWMR1', 'DBWMR10', 'DBWMR11', 'DBWMR12', 'DBWMR13', 'DBWMR14', 'DBWMR15', 'DBWMR16', 'DBWMR17', 'DBWMR18', 'DBWMR19', 'DBWMR2', 'DBWMR20', 'DBWMR21', 'DBWMR22', 'DBWMR23', 'DBWMR24', 'DBWMR3', 'DBWMR4', 'DBWMR5', 'DBWMR6', 'DBWMR7', 'DBWMR8', 'DBWMR9', 'HYP', 'I1L', 'I1R', 'I2L', 'I2R', 'I3', 'I4', 'I5', 'I6', 'INTESTINE', 'INTL', 'INTR', 'M1', 'M2L', 'M2R', 'M3L', 'M3R', 'M4', 'M5', 'MC1DL', 'MC1DR', 'MC1V', 'MC2DL', 'MC2DR', 'MC2V', 'MC3DL', 'MC3DR', 'MC3V', 'MCL', 'MCR', 'MI', 'NSML', 'NSMR', 'PM1D', 'PM1VL', 'PM1VR', 'PM2D', 'PM2VL', 'PM2VR', 'PM3D', 'PM3VL', 'PM3VR', 'PM4D', 'PM4VL', 'PM4VR', 'PM5D', 'PM5VL', '

In [3]:
cook_Gg, cook_Gs = load_connectome_cook()